In [7]:
# setup libraries
import numpy as np
import tensorflow as tf


In [17]:
data = "hello"
char_set = sorted(list(set(data)))
def one_hot_encode(char):
    return np.array([int(char == char_set[i]) for i in range(len(char_set))])

print(one_hot_encode("h"))

[0 1 0 0]


In [ ]:
import tensorflow.compat.v1.nn as nn 

In [19]:
cell = nn.rnn_cell.BasicLSTMCell(2)
x_data = np.array([[[0,1,0,0]]], dtype=np.float32)
outputs, _states = nn.dynamic_rnn(cell, x_data, dtype=tf.float32)
print(outputs)


tf.Tensor([[[0.06630785 0.09286073]]], shape=(1, 1, 2), dtype=float32)


/var/folders/4r/3qmgzxs52t570_hcxwnccnl40000gq/T/ipykernel_89142/2185952186.py:1: UserWarning: `tf.nn.rnn_cell.BasicLSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  cell = nn.rnn_cell.BasicLSTMCell(2)


In [22]:
def generate_input(source):
    return np.array([one_hot_encode(char) for char in source], dtype=np.float32)

tf.expand_dims(generate_input("hello"), axis=0)

<tf.Tensor: shape=(1, 5, 4), dtype=float32, numpy=
array([[[0., 1., 0., 0.],
        [1., 0., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]]], dtype=float32)>

In [23]:
hidden_size = 2
sequence_length = len("hello")
cell = nn.rnn_cell.BasicLSTMCell(hidden_size)
x_data = tf.expand_dims(generate_input("hello"), axis=0)
outputs, _states = nn.dynamic_rnn(cell, x_data, dtype=tf.float32)
print(outputs)


tf.Tensor(
[[[ 0.07471338  0.09115146]
  [ 0.15714878  0.13329597]
  [ 0.20319963 -0.00927099]
  [ 0.2341277  -0.17273603]
  [ 0.17550129 -0.12324703]]], shape=(1, 5, 2), dtype=float32)


/var/folders/4r/3qmgzxs52t570_hcxwnccnl40000gq/T/ipykernel_89142/325307102.py:3: UserWarning: `tf.nn.rnn_cell.BasicLSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  cell = nn.rnn_cell.BasicLSTMCell(hidden_size)


In [25]:
data = "hihello"
char_set = sorted(list(set(data)))

def one_hot_encode(char):
    return np.array([int(char == char_set[i]) for i in range(len(char_set))])

def generate_input(source):
    return np.array([one_hot_encode(char) for char in source], dtype=np.float32)

tf.expand_dims(generate_input("hihello"), axis=0)

<tf.Tensor: shape=(1, 7, 5), dtype=float32, numpy=
array([[[0., 1., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 1., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0.],
        [0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 1.]]], dtype=float32)>

In [29]:
import tensorflow_addons as tfa
y_data = tf.constant([[1,1,1]])
prediction1 = tf.constant([[[0.3,0.7],[0.3,0.7],[0.3,0.7]]], dtype=tf.float32)
prediction2 = tf.constant([[[0.1, 0.9],[0.1, 0.9],[0.1, 0.9]]], dtype=tf.float32)

weights = tf.constant([[1,1,1]], dtype=tf.float32)

sequence_loss1 = tfa.seq2seq.sequence_loss(prediction1, y_data, weights)
sequence_loss2 = tfa.seq2seq.sequence_loss(prediction2, y_data, weights)
print("Loss1: ", sequence_loss1.numpy()
, "Loss2: ", sequence_loss2.numpy())

Loss1:  0.5130153 Loss2:  0.3711007


/Users/jerrykim/workspace/deep-learning/tensorflow/venv/lib/python3.11/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [66]:
import tensorflow as tf
import numpy as np
import tensorflow.compat.v1.nn as nn
import tensorflow_addons as tfa
import keras

tf.random.set_seed(777)  # reproducibility
sample = " if you want you"
charset = sorted(list(set(sample)))  # index -> char
char2idx = {c: i for i, c in enumerate(charset)}  # char -> index

# hyper parameters
dic_size = len(char2idx)  # RNN input size (one hot size)
hidden_size = len(char2idx)  # RNN output size
num_classes = len(char2idx)  # final output size (RNN or softmax, etc.)
batch_size = 1  # one sample data, one batch
sequence_length = len(sample) - 1  # number of lstm rollings (unit #)
learning_rate = 0.1

sample_idx = [char2idx[c] for c in sample]  # char to index
# x_data = [sample_idx[:-1]]  # X data sample (0 ~ n-1) hello: hell
# y_data = [sample_idx[1:]]   # Y label sample (1 ~ n) hello: ello

def one_hot_encode(char):
    return np.array([int(char == charset[i]) for i in range(len(charset))])

def generate_input(source):
    return np.array([one_hot_encode(char) for char in source], dtype=np.float32)

x_data = tf.expand_dims(generate_input(sample[:-1]), axis=0)
y_data = tf.expand_dims(generate_input(sample[1:]), axis=0)

cell = keras.layers.LSTMCell(hidden_size)
initial_state = cell.get_initial_state(batch_size=batch_size, dtype=tf.float32)
# print(initial_state)
weights = tf.ones([batch_size, sequence_length])
#print(weights)
for i in range(20):
    with tf.GradientTape() as tape:
        outputs, _states = nn.dynamic_rnn(cell, x_data, initial_state=initial_state, dtype=tf.float32)
        # print(outputs)
        sequence_loss = tfa.seq2seq.sequence_loss(logits=outputs, targets=y_data, weights=weights)
        mean_loss = tf.reduce_mean(sequence_loss)
        print("step: ", i, " loss: ", mean_loss.numpy())
    grads = tape.gradient(mean_loss, cell.trainable_variables)
    optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=learning_rate)
    optimizer.apply_gradients(zip(grads, cell.trainable_variables))
    prediction = tf.argmax(outputs, axis=2)    
    result_str = [charset[c] for c in np.squeeze(prediction)]
    # print("prediction: ", prediction.numpy())
    print("Prediction str: ", ''.join(result_str))



tf.Tensor(
[[[ 0.01321007  0.04635428  0.07408268 -0.02713791 -0.02591958
   -0.04962373 -0.0792703  -0.03478202 -0.04754604  0.01435794]
  [-0.04658528  0.00550043  0.04283317 -0.07995345 -0.05623904
   -0.08495228  0.01850726 -0.04871839 -0.09588163  0.06942505]
  [ 0.03590014  0.01109139 -0.00804782  0.00051419  0.03101098
   -0.03008408 -0.05675561  0.03510022 -0.10037988  0.00074052]
  [ 0.03984265  0.05516316  0.07211756 -0.03327267  0.00567346
   -0.07272393 -0.11708515 -0.00935009 -0.1341738   0.00994467]
  [-0.02587041  0.08274609  0.0717361  -0.07030126  0.02432912
   -0.09829973 -0.08974345  0.01033309 -0.08339881  0.04034544]
  [-0.05861273  0.09977069  0.06624457 -0.002056    0.03907097
   -0.09859533  0.00041083  0.03706269 -0.11410582 -0.07426126]
  [ 0.02469691  0.02738264  0.09286562 -0.02815435 -0.02580206
    0.00609496  0.05143228 -0.02521962 -0.08076978 -0.11420061]
  [ 0.03080313  0.06444271  0.13317785 -0.05494668 -0.04345125
   -0.03083249 -0.04363571 -0.0405684

In [73]:
import tensorflow as tf
import numpy as np
import tensorflow.compat.v1.nn as nn
import tensorflow_addons as tfa
import keras

sentence = ("if you want to build a ship, don't drum up people together to "
            "collect wood and don't assign them tasks and work, but rather "
            "teach them to long for the endless immensity of the sea. - Antoine ")


char_set = sorted(list(set(sentence)))
char2idx = {c: i for i, c in enumerate(char_set)}  # char -> index

def one_hot_encode(char):
    return np.array([int(char == char_set[i]) for i in range(len(char_set))])

def generate_code(source):
    return tf.expand_dims(np.array([one_hot_encode(char) for char in source], dtype=np.float32), axis=0)

sequence_length = 7
data_dim = len(char_set)
hidden_size = len(char_set)
num_classes = len(char_set)

dataX = []
dataY = []
tf.random.set_seed(777)

for i in range(0, len(sentence) - sequence_length):
    x_str = sentence[i:i + sequence_length]
    y_str = sentence[i + 1: i + sequence_length + 1]
    # print(i, x_str, '->', y_str)

    # x = [char2idx[c] for c in x_str]  # x str to index
    # y = [char2idx[c] for c in y_str]  # y str to index
    
    dataX.append(generate_code(x_str))
    dataY.append(generate_code(y_str))

X_data = np.array(dataX)
Y_data = np.array(dataY)


# print(Y_data.shape)
batch_size = len(X_data)
# print(batch_size)
cell = keras.layers.LSTMCell(hidden_size)
initial_state = cell.get_initial_state(batch_size=batch_size, dtype=tf.float32)
print(initial_state)
weights = tf.ones([batch_size, sequence_length])

for i in range(1):
    with tf.GradientTape() as tape:
        outputs, _states = nn.dynamic_rnn(cell, X_data, initial_state=initial_state, dtype=tf.float32)        
        # print(outputs)
        sequence_loss = tfa.seq2seq.sequence_loss(logits=outputs, targets=Y_data, weights=weights)
        mean_loss = tf.reduce_mean(sequence_loss)
        print("step: ", i, " loss: ",sequence_loss)

[<tf.Tensor: shape=(184, 27), dtype=float32, numpy=
array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)>, <tf.Tensor: shape=(184, 27), dtype=float32, numpy=
array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)>]


InvalidArgumentError: Exception encountered when calling layer 'lstm_cell_30' (type LSTMCell).

{{function_node __wrapped__AddV2_device_/job:localhost/replica:0/task:0/device:CPU:0}} Incompatible shapes: [184,7,108] vs. [184,108] [Op:AddV2] name: 

Call arguments received by layer 'lstm_cell_30' (type LSTMCell):
  • inputs=tf.Tensor(shape=(184, 7, 27), dtype=float32)
  • states=['tf.Tensor(shape=(184, 27), dtype=float32)', 'tf.Tensor(shape=(184, 27), dtype=float32)']
  • training=None